In [ ]:
# n = 5000
# # sample y from a Gaussian Process
# true_length_scale = 0.1
# true_variance = 2
# true_noise_variance = 0.1
# def sample_gp(x, mean_func, cov_func, num_samples=1, length_scale=1.0, variance=1.0):
#     """
#     Sample from a Gaussian Process.
    
#     Parameters:
#     x : np.ndarray
#         Input points where the GP is evaluated.
#     mean_func : callable
#         Mean function of the GP.
#     cov_func : callable
#         Covariance function of the GP.
#     num_samples : int
#         Number of samples to draw from the GP.
    
#     Returns:
#     np.ndarray
#         Samples from the GP at the input points x.
#     """
#     # Compute the mean and covariance matrix
#     mean = mean_func(x)
#     cov = cov_func(x, x, length_scale=true_length_scale, variance=true_variance)
    
#     # Draw samples from the multivariate normal distribution
#     samples = torch.tensor(np.random.multivariate_normal(mean, cov, num_samples))
    
#     return samples.T  # Transpose to get samples along columns
# # Example mean and covariance functions
# def mean_func(x):
#     return torch.zeros_like(x)
# def cov_func(x1, x2,length_scale=1.0,variance=1.0):
#     return torch.tensor(variance*np.exp((-0.5 * np.subtract.outer(x1, x2)**2)/length_scale**2))
# # Generate input points




# # x = torch.linspace(-1, 1, n, dtype=torch.float64)
# # Sample from the GP
# # y = sample_gp(x, mean_func, cov_func, num_samples=1,length_scale=true_length_scale, variance=true_variance)
# # y = y.to(dtype=torch.float64).flatten()+ true_noise_variance * torch.randn(n, dtype=torch.float64)
# # Plot the samples
# # plt.figure(figsize=(10, 5))
# # plt.plot(x, y)
# # plt.title("Samples from a Gaussian Process, parameters: length_scale=%.2f, variance=%.2f, noise_variance=%.2f" % (true_length_scale, true_variance, true_noise_variance))


In [ ]:



eps = EPSILON
trace_samples =10
sigmasq = torch.tensor(0.1, dtype=torch.float64)  # noise variance
init_kernel
x_new = x 
num_iters = 1
# def efgp1d_gradient(x, y, sigmasq, kernel,eps,trace_samples):
# x = x.to(dtype=torch.float64).flatten()   # shape: (N,)
y = y.to(dtype=torch.float64).flatten()     # shape: (N,)
x0, x1 = torch.min(x), torch.max(x)
# opts = {'get_var': True, 'get_log_marginal_likelihood': True}
L = x1 - x0
N = x.shape[0]
# eps = EPSILON
# eps = 1e-20

# Get Fourier frequencies and weights.
xis, h, mtot = get_xis(kernel, eps, L)  # Expect len(xis)=M (e.g., M=97)
# Compute weights: ws = sqrt(S(xis)*h)
ws = torch.sqrt(kernel.spectral_density(xis) * h)  # ws: (M,)   # will change to h^d 
D = torch.diag(ws).to(dtype=torch.complex128)         # D: (M, M)

# Form design features F (size: (N, M)).
#TODO : use UFFTConv1d instead of this
F = torch.exp(1j * 2 * math.pi * torch.outer(x, xis)).to(dtype=torch.complex128)
#TODO higher dim 
# F = torch.exp(1j * 2 * torch.pi * (x@xis.t()).to(dtype=torch.complex128))

v, j_indices = compute_convolution_vector_vectorized(m=int((mtot - 1) / 2), x=x, h=h)
Afun = lambda beta: D @ FFTConv1d(v, D @ beta)() + sigmasq * beta


# A = D @ (torch.conj(F).T @ F) @ D  # A: (M, M)

# -------------------
# Compute term 2: easy way
# rhs = D @ torch.conj(F).T @ y.to(dtype=torch.complex128)  # rhs: (M,)


#via NUFFT 
xcen = (x1+x0)/2
tphx = 2*math.pi*h*(x - xcen)
# tphxtrgs = 2*math.pi*h*(x - xcen)


nuffttol = EPSILON/20
isign = -1
fadjbeta = lambda beta: torch.tensor(finufft.nufft1d1(x=tphx.numpy(), c=beta.to(torch.complex128).numpy(),isign= isign, eps= nuffttol, n_modes=mtot))
rhs = fadjbeta(y)
# rhs = finufft.nufft1d1(x=tphx.numpy(), c=y.to(torch.complex128).numpy(),isign= isign, eps= nuffttol, n_modes=mtot)
rhs = ws * rhs;       

## want F^* alpha

# cg_object = ConjugateGradients(
#     A_apply_function=A + sigmasq * torch.eye(mtot, dtype=A.dtype),
#     b=rhs,
#     x0=torch.zeros_like(rhs),
#     early_stopping=opts.get('early_stopping', False)
# )
cg_object = ConjugateGradients(
    A_apply_function=Afun,
    b=rhs,
    x0=torch.zeros_like(rhs),
    early_stopping=False
    # early_stopping=opts.get('early_stopping', False)
)
beta = cg_object.solve()  # beta: (M,)
alpha = 1/sigmasq * (y - F @ D @ beta)  # alpha: (N,)

# -------------------
Dprime = h * kernel.spectral_grad(xis)  # shape: (M, n_params)
# This yields the derivative of the diagonal of D^2, since D^2 = diag(S(xis)*h).

# --- Embed each column of Dprime as a diagonal matrix.
# Dprime is (M, n_params); we want each column (length M) as an (M,M) diagonal.
# Transpose so each row corresponds to one hyperparameter:
Dprime_t = Dprime.transpose(0, 1)   # shape: (n_params, M)
# Embed each row into a diagonal matrix -> (n_params, M, M)
D_diag_temp = torch.diag_embed(Dprime_t).to(dtype=torch.complex128)  # shape: (n_params, M, M)

# -------------------
# Compute khat_prime for the kernel hyperparameters:
# For each hyperparameter i, we want:
#     khat_prime_i = F * diag(Dprime_i) * F^*
# Expand F and its conjugate-transpose to include a batch dimension.
# TODO : use nufft 
N_val, M_val = F.shape  # M_val should equal M.
F_expand = F.unsqueeze(0).expand(D_diag_temp.shape[0], N_val, M_val)  # shape: (n_params, N, M)
F_adjoint = F.conj().transpose(-2, -1)  # shape: (M, N)
F_adjoint_expand = F_adjoint.unsqueeze(0).expand(D_diag_temp.shape[0], M_val, N_val)  # shape: (n_params, M, N)
# Batched multiplication:

## FD' F^* 
khat_prime_kernel = torch.bmm(torch.bmm(F_expand, D_diag_temp), F_adjoint_expand)  # shape: (n_params, N, N)

# Append the noise derivative (for noise, d/dσ² (K+σ²I) = I) as an extra hyperparameter.
noise_derivative = torch.eye(N, dtype=khat_prime_kernel.dtype).unsqueeze(0)  # shape: (1, N, N)
khat_prime_kernel_full = torch.cat([khat_prime_kernel, noise_derivative], dim=0)  # shape: (n_params+1, N, N)
# Optionally, permute so that the hyperparameter dimension is last:
khat_prime = khat_prime_kernel_full.permute(1, 2, 0)  # shape: (N, N, n_params+1)

# -------------------
# Compute term 2 (quadratic term) for each hyperparameter:
# For hyperparameter i, term2_i = α^* (dK/dθ_i) α.
term2 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128)
for i in range(khat_prime.shape[2]-1):
    ## alpha^* F D' F^* alpha
    fadjalpha = fadjbeta(alpha)
    term2[i] = fadjalpha.conj()@ (Dprime[:,i]*fadjalpha)
# term2[i] = torch.matmul(alpha.conj().unsqueeze(0),
#                         torch.matmul(khat_prime[:, :, i], alpha.unsqueeze(1))).squeeze()
term2[-1] = alpha.conj() @ alpha  # noise term
# -------------------
# Compute term 1 (trace term) via Monte Carlo.
# trace_samples = 10
# try with rademachers instead..


# Z = torch.distributions.multivariate_normal.MultivariateNormal(
#         loc=torch.zeros_like(x),
#         covariance_matrix=torch.eye(N, dtype=torch.float64)
#     ).sample([trace_samples])  # shape: (trace_samples, N)
# sample Z iid rademacher variables shape (trace_samples, N)
Z = torch.randint(0, 2, (trace_samples, N), dtype=torch.float64) * 2 - 1

term1 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128)
for i in range(khat_prime.shape[2]):  # for each hyperparameter derivative
    sum_term1 = 0.0 + 0.0j
#TODO: fix the apply function here 
    for j in range(trace_samples):
        z_sample = Z[j, :].to(dtype=torch.complex128)  # shape: (N,)
        temp_rhs = khat_prime[:, :, i] @ z_sample.unsqueeze(1)  # shape: (N, 1)
        rhs_sample = D @ torch.conj(F).T @ temp_rhs  # shape: (M, 1)
        cg_object = ConjugateGradients(
            A_apply_function=Afun,
            b=rhs_sample.squeeze(),  # shape: (M,)
            x0=torch.zeros_like(rhs_sample.squeeze()),
            early_stopping=False
        )
        beta_sample = cg_object.solve()  # shape: (M,)
        # alpha_sample = 1/sigmasq * (y - F @ D @ beta_sample)  # shape: (N,)
        alpha_sample = 1/sigmasq * (temp_rhs.flatten() - F @ D @ beta_sample)

        sum_term1 += torch.dot(z_sample, alpha_sample)
    term1[i] = sum_term1 / trace_samples

# -------------------
# Final gradient for each hyperparameter:
# grad_i = 0.5 * [ term1_i - term2_i ]
grad = 0.5 * (term1 - term2)
print(term2.real)
    # return grad.real
    # save the gradients for each loop
    # if iter == 0:
    #     grad_all = grad
    # else:
    #     grad_all = torch.cat([grad_all, grad], dim=0)
    
    # Print the gradients.
# grad = efgp1d_gradient(x, y, sigmasq, kernel,EPSILON,trace_samples=10)
# for i in range(grad.shape[0]):
#     print(f"Hyperparameter {i}: gradient approx = {grad[i].real.item():.6f}")


In [ ]:
import torch
import time
import math
from efgp1d import compute_convolution_vector_vectorized, FFTConv1d, BatchFFTConv1d



sigmasq = torch.tensor(0.1, dtype=torch.float64)  # noise variance
init_kernel
x_new = x 
num_iters = 1
def efgp1d_gradient(x, y, sigmasq, kernel,eps,trace_samples):
    # x = x.to(dtype=torch.float64).flatten()   # shape: (N,)
    y = y.to(dtype=torch.float64).flatten()     # shape: (N,)
    x0, x1 = torch.min(x), torch.max(x)
    # opts = {'get_var': True, 'get_log_marginal_likelihood': True}
    L = x1 - x0
    N = x.shape[0]
    # eps = EPSILON
    # eps = 1e-20

    # Get Fourier frequencies and weights.
    xis, h, mtot = get_xis(kernel, eps, L)  # Expect len(xis)=M (e.g., M=97)
    # Compute weights: ws = sqrt(S(xis)*h)
    ws = torch.sqrt(kernel.spectral_density(xis) * h)  # ws: (M,)   # will change to h^d 
    D = torch.diag(ws).to(dtype=torch.complex128)         # D: (M, M)
    
    # Form design features F (size: (N, M)).
    #TODO : use UFFTConv1d instead of this
    F = torch.exp(1j * 2 * math.pi * torch.outer(x, xis)).to(dtype=torch.complex128)
    #TODO higher dim 
    # F = torch.exp(1j * 2 * torch.pi * (x@xis.t()).to(dtype=torch.complex128))

    v, j_indices = compute_convolution_vector_vectorized(m=int((mtot - 1) / 2), x=x, h=h)
    Afun = lambda beta: D @ FFTConv1d(v, D @ beta)() + sigmasq * beta


    # A = D @ (torch.conj(F).T @ F) @ D  # A: (M, M)
    
    # -------------------
    # Compute term 2: easy way
    # rhs = D @ torch.conj(F).T @ y.to(dtype=torch.complex128)  # rhs: (M,)


    #via NUFFT 
    xcen = (x1+x0)/2
    tphx = 2*math.pi*h*(x - xcen)
    # tphxtrgs = 2*math.pi*h*(x - xcen)

    
    nuffttol = EPSILON/20
    isign = -1
    rhs = finufft.nufft1d1(x=tphx.numpy(), c=y.to(torch.complex128).numpy(),isign= isign, eps= nuffttol, n_modes=mtot)
    rhs = ws * rhs;       



    # cg_object = ConjugateGradients(
    #     A_apply_function=A + sigmasq * torch.eye(mtot, dtype=A.dtype),
    #     b=rhs,
    #     x0=torch.zeros_like(rhs),
    #     early_stopping=opts.get('early_stopping', False)
    # )
    cg_object = ConjugateGradients(
        A_apply_function=Afun,
        b=rhs,
        x0=torch.zeros_like(rhs),
        early_stopping=False
        # early_stopping=opts.get('early_stopping', False)
    )
    beta = cg_object.solve()  # beta: (M,)
    alpha = 1/sigmasq * (y - F @ D @ beta)  # alpha: (N,)
    
    # -------------------
    Dprime = h * kernel.spectral_grad(xis)  # shape: (M, n_params)
    # This yields the derivative of the diagonal of D^2, since D^2 = diag(S(xis)*h).
    
    # --- Embed each column of Dprime as a diagonal matrix.
    # Dprime is (M, n_params); we want each column (length M) as an (M,M) diagonal.
    # Transpose so each row corresponds to one hyperparameter:
    Dprime_t = Dprime.transpose(0, 1)   # shape: (n_params, M)
    # Embed each row into a diagonal matrix -> (n_params, M, M)
    D_diag_temp = torch.diag_embed(Dprime_t).to(dtype=torch.complex128)  # shape: (n_params, M, M)
    
    # -------------------
    # Compute khat_prime for the kernel hyperparameters:
    # For each hyperparameter i, we want:
    #     khat_prime_i = F * diag(Dprime_i) * F^*
    # Expand F and its conjugate-transpose to include a batch dimension.
    # TODO : use nufft 
    N_val, M_val = F.shape  # M_val should equal M.
    F_expand = F.unsqueeze(0).expand(D_diag_temp.shape[0], N_val, M_val)  # shape: (n_params, N, M)
    F_adjoint = F.conj().transpose(-2, -1)  # shape: (M, N)
    F_adjoint_expand = F_adjoint.unsqueeze(0).expand(D_diag_temp.shape[0], M_val, N_val)  # shape: (n_params, M, N)
    # Batched multiplication:

    ## FD' F^* 
    khat_prime_kernel = torch.bmm(torch.bmm(F_expand, D_diag_temp), F_adjoint_expand)  # shape: (n_params, N, N)
    
    # Append the noise derivative (for noise, d/dσ² (K+σ²I) = I) as an extra hyperparameter.
    noise_derivative = torch.eye(N, dtype=khat_prime_kernel.dtype).unsqueeze(0)  # shape: (1, N, N)
    khat_prime_kernel_full = torch.cat([khat_prime_kernel, noise_derivative], dim=0)  # shape: (n_params+1, N, N)
    # Optionally, permute so that the hyperparameter dimension is last:
    khat_prime = khat_prime_kernel_full.permute(1, 2, 0)  # shape: (N, N, n_params+1)
    
    # -------------------
    # Compute term 2 (quadratic term) for each hyperparameter:
    # For hyperparameter i, term2_i = α^* (dK/dθ_i) α.
    term2 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128)
    for i in range(khat_prime.shape[2]):
        ## alpha^* F D' F^* alpha
        term2[i] = torch.matmul(alpha.conj().unsqueeze(0),
                                torch.matmul(khat_prime[:, :, i], alpha.unsqueeze(1))).squeeze()
    
    # -------------------
    # Compute term 1 (trace term) via Monte Carlo.
    # trace_samples = 10
    # try with rademachers instead..

    
    # Z = torch.distributions.multivariate_normal.MultivariateNormal(
    #         loc=torch.zeros_like(x),
    #         covariance_matrix=torch.eye(N, dtype=torch.float64)
    #     ).sample([trace_samples])  # shape: (trace_samples, N)
    # sample Z iid rademacher variables shape (trace_samples, N)
    Z = torch.randint(0, 2, (trace_samples, N), dtype=torch.float64) * 2 - 1
    
    term1 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128)
    for i in range(khat_prime.shape[2]):  # for each hyperparameter derivative
        sum_term1 = 0.0 + 0.0j
#TODO: fix the apply function here 
        for j in range(trace_samples):
            z_sample = Z[j, :].to(dtype=torch.complex128)  # shape: (N,)
            temp_rhs = khat_prime[:, :, i] @ z_sample.unsqueeze(1)  # shape: (N, 1)
            rhs_sample = D @ torch.conj(F).T @ temp_rhs  # shape: (M, 1)
            cg_object = ConjugateGradients(
                A_apply_function=Afun,
                b=rhs_sample.squeeze(),  # shape: (M,)
                x0=torch.zeros_like(rhs_sample.squeeze()),
                early_stopping=False
            )
            beta_sample = cg_object.solve()  # shape: (M,)
            # alpha_sample = 1/sigmasq * (y - F @ D @ beta_sample)  # shape: (N,)
            alpha_sample = 1/sigmasq * (temp_rhs.flatten() - F @ D @ beta_sample)

            sum_term1 += torch.dot(z_sample, alpha_sample)
        term1[i] = sum_term1 / trace_samples

    # -------------------
    # Final gradient for each hyperparameter:
    # grad_i = 0.5 * [ term1_i - term2_i ]
    grad = 0.5 * (term1 - term2)
    # print(term2.real)
    return grad.real
    # save the gradients for each loop
    # if iter == 0:
    #     grad_all = grad
    # else:
    #     grad_all = torch.cat([grad_all, grad], dim=0)
    
    # Print the gradients.
grad = efgp1d_gradient(x, y, sigmasq, kernel,EPSILON,trace_samples=10)
for i in range(grad.shape[0]):
    print(f"Hyperparameter {i}: gradient approx = {grad[i].real.item():.6f}")


In [ ]:
Z = Z.to(dtype=torch.complex128)  # cast to complex for consistency

# Define a batched operator based on FFTConv1d for use in the CG solver.
def Afun_batch(beta):
    """
    Batched operator acting on beta of shape (B, M).
    Returns: ws * FFTConv1d(v, ws * beta)() + sigmasq * beta, shape (B, M).
    """
    # Multiply beta by ws elementwise (equivalent to a diagonal D multiplication)
    D_beta = ws * beta
    # FFTConv1d expects the second argument to have shape (B, L); ensure beta is batched.
    conv_result = BatchFFTConv1d(v, D_beta)()  # Output shape: (B, M) if beta is (B, M)
    return ws * conv_result + sigmasq * beta

# We'll now loop over each hyperparameter derivative index;
# for each, we form a batched right-hand side (one per trace sample), solve the linear systems in batch,
# and compute the corresponding trace term.
term1 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128, device=khat_prime.device)

for i in range(khat_prime.shape[2]):
    # For the i-th hyperparameter derivative, get A_i = (khat_prime)_i  of shape (N, N).
    A_i = khat_prime[:, :, i]  # (N, N)
    # Expand A_i into a batch to multiply with each Rademacher sample.
    A_i_batch = A_i.unsqueeze(0).expand(trace_samples, -1, -1)  # (trace_samples, N, N)
    
    # Reshape Z so that each sample is a column vector.
    z_batch = Z.unsqueeze(-1)  # (trace_samples, N, 1)
    # Compute the right-hand side for each trace sample: temp_rhs = A_i @ z.

    ## khat_prime[:, :, i] @ z_batch
    temp_rhs_batch = torch.bmm(A_i_batch, z_batch)  # (trace_samples, N, 1)
    
    # Next, form the right-hand side for the linear system:
    # b_batch = ws * (F^H @ temp_rhs).
    # Compute F^H @ temp_rhs for each sample.
    # Note: F^H = conj(F)^T. temp_rhs_batch: (trace_samples, N, 1), so we need to reshape F^H.


    ## solve (FDF^*)^{-1} FD' F^* zk
    inner = torch.matmul(temp_rhs_batch.squeeze(-1), torch.conj(F))  # (trace_samples, M)
    b_batch = ws * inner  # (trace_samples, M)
    
    # Solve A x = b for all trace samples using the Batch CG solver.
    cg_solver = BatchConjugateGradients(
        A_apply_function=Afun_batch,
        b=b_batch,
        x0=torch.zeros_like(b_batch),
        tol=1e-6,
        early_stopping=False
    )
    beta_batch = cg_solver.solve()  # (trace_samples, M)
    
    # Compute F @ (ws * beta_batch) for each trace sample.
    D_beta_batch = ws * beta_batch  # (trace_samples, M)
    F_expanded = F.unsqueeze(0).expand(trace_samples, -1, -1)  # (trace_samples, N, M)
    F_beta = torch.bmm(F_expanded, D_beta_batch.unsqueeze(-1)).squeeze(-1)  # (trace_samples, N)
    
    # Compute the corresponding alpha for each trace sample:
    # alpha_batch = 1/sigmasq * (temp_rhs - F @ (ws * beta_batch))
    temp_rhs_flat = temp_rhs_batch.squeeze(-1)  # (trace_samples, N)
    alpha_batch = 1/sigmasq * (temp_rhs_flat - F_beta)  # (trace_samples, N)
    
    # Now, compute the dot product z^T * alpha for each trace sample and average.
    dots = torch.sum(Z * alpha_batch, dim=1)  # (trace_samples,)
    term1[i] = torch.mean(dots)

print(term1.real)


# batched without nufft

In [ ]:
# def efgp1d_gradient_batched(x, y, sigmasq, kernel,eps,trace_samples):


#     # Ensure x and y are float64 and flattened.
#     x = x.to(dtype=torch.float64).flatten()   # shape: (N,)
#     y = y.to(dtype=torch.float64).flatten()     # shape: (N,)
#     x0, x1 = torch.min(x), torch.max(x)
#     L = x1 - x0
#     N = x.shape[0]

#     # Get Fourier frequencies and quadrature weights.
#     xis, h, mtot = get_xis(kernel, eps, L)  # e.g. len(xis)==M, where M=97 for example.

#     # Compute weights: ws = sqrt(S(xis) * h)
#     ws = torch.sqrt(kernel.spectral_density(xis) * h)  # shape: (M,)
#     # We represent D as a diagonal using ws.
#     # Convert to complex type.
#     ws = ws.to(dtype=torch.complex128)

#     # Build design matrix features F: shape (N, M).
#     F = torch.exp(1j * 2 * math.pi * torch.outer(x, xis)).to(dtype=torch.complex128)

#     # Compute convolution vector v (and indices, if needed).
#     v, j_indices = compute_convolution_vector_vectorized(m=int((mtot - 1) / 2), x=x, h=h)
#     v = v.to(dtype=torch.complex128)

#     # Define the operator for the main linear system.
#     # Here, we use FFTConv1d with kernel v and treat D as a diagonal multiplication by ws.
#     Afun = lambda beta: ws * BatchFFTConv1d(v, ws * beta)() + sigmasq * beta

#     # Solve the main system with your (non-batched) conjugate gradient solver.
#     rhs = ws * (torch.conj(F).T @ y.to(dtype=torch.complex128))  # shape: (M,)
#     cg_object = ConjugateGradients(
#         A_apply_function=Afun,
#         b=rhs,
#         x0=torch.zeros_like(rhs),
#         early_stopping=False
#     )
#     beta = cg_object.solve()  # beta: (M,)
#     alpha = 1/sigmasq * (y - F @ (ws * beta))  # shape: (N,)

#     ##############################################
#     # Compute Hyperparameter Gradients
#     ##############################################
#     # 1. Compute the derivative of the diagonal of D².
#     Dprime = h * kernel.spectral_grad(xis)  # shape: (M, n_params)
#     # Embed each column as a diagonal matrix.
#     Dprime_t = Dprime.transpose(0, 1)  # shape: (n_params, M)
#     D_diag_temp = torch.diag_embed(Dprime_t).to(dtype=torch.complex128)  # shape: (n_params, M, M)

#     # 2. Compute khat_prime for kernel hyperparameters:
#     #    khat_prime = F @ diag(Dprime_i) @ F^H for each hyperparameter i.
#     N_val, M_val = F.shape  # (N, M)
#     F_expand = F.unsqueeze(0).expand(D_diag_temp.shape[0], N_val, M_val)  # (n_params, N, M)
#     F_adjoint = torch.conj(F).transpose(-2, -1)  # (M, N)
#     F_adjoint_expand = F_adjoint.unsqueeze(0).expand(D_diag_temp.shape[0], M_val, N_val)  # (n_params, M, N)
#     khat_prime_kernel = torch.bmm(torch.bmm(F_expand, D_diag_temp), F_adjoint_expand)  # (n_params, N, N)
#     # Append the noise derivative (for σ², the derivative is I)
#     noise_derivative = torch.eye(N, dtype=khat_prime_kernel.dtype, device=khat_prime_kernel.device).unsqueeze(0)  # (1, N, N)
#     khat_prime_kernel_full = torch.cat([khat_prime_kernel, noise_derivative], dim=0)  # (n_params+1, N, N)
#     # Permute so that hyperparameter derivative dimension is last.
#     khat_prime = khat_prime_kernel_full.permute(1, 2, 0)  # shape: (N, N, n_params+1)

#     # 3. Compute the quadratic form term2 for each hyperparameter:
#     term2 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128, device=khat_prime.device)
#     for i in range(khat_prime.shape[2]):
#         term2[i] = (torch.conj(alpha).unsqueeze(0) @ (khat_prime[:, :, i] @ alpha.unsqueeze(1))).squeeze()

#     ##############################################
#     # Monte Carlo Trace Term (term1) using Batch CG solver
#     ##############################################
#     # Create trace_samples Rademacher vectors. (Each entry is +1 or -1.)
#     Z = torch.randint(0, 2, (trace_samples, N), dtype=torch.float64, device=x.device) * 2 - 1  # shape: (trace_samples, N)
#     Z = Z.to(dtype=torch.complex128)  # cast to complex for consistency

#     # Define a batched operator based on FFTConv1d for use in the CG solver.
#     def Afun_batch(beta):
#         """
#         Batched operator acting on beta of shape (B, M).
#         Returns: ws * FFTConv1d(v, ws * beta)() + sigmasq * beta, shape (B, M).
#         """
#         # Multiply beta by ws elementwise (equivalent to a diagonal D multiplication)
#         D_beta = ws * beta
#         # FFTConv1d expects the second argument to have shape (B, L); ensure beta is batched.
#         conv_result = BatchFFTConv1d(v, D_beta)()  # Output shape: (B, M) if beta is (B, M)
#         return ws * conv_result + sigmasq * beta

#     # We'll now loop over each hyperparameter derivative index;
#     # for each, we form a batched right-hand side (one per trace sample), solve the linear systems in batch,
#     # and compute the corresponding trace term.
#     term1 = torch.zeros(khat_prime.shape[2], dtype=torch.complex128, device=khat_prime.device)

#     for i in range(khat_prime.shape[2]):
#         # For the i-th hyperparameter derivative, get A_i = (khat_prime)_i  of shape (N, N).
#         A_i = khat_prime[:, :, i]  # (N, N)
#         # Expand A_i into a batch to multiply with each Rademacher sample.
#         A_i_batch = A_i.unsqueeze(0).expand(trace_samples, -1, -1)  # (trace_samples, N, N)
        
#         # Reshape Z so that each sample is a column vector.
#         z_batch = Z.unsqueeze(-1)  # (trace_samples, N, 1)
#         # Compute the right-hand side for each trace sample: temp_rhs = A_i @ z.
#         temp_rhs_batch = torch.bmm(A_i_batch, z_batch)  # (trace_samples, N, 1)
        
#         # Next, form the right-hand side for the linear system:
#         # b_batch = ws * (F^H @ temp_rhs).
#         # Compute F^H @ temp_rhs for each sample.
#         # Note: F^H = conj(F)^T. temp_rhs_batch: (trace_samples, N, 1), so we need to reshape F^H.
#         inner = torch.matmul(temp_rhs_batch.squeeze(-1), torch.conj(F))  # (trace_samples, M)
#         b_batch = ws * inner  # (trace_samples, M)
        
#         # Solve A x = b for all trace samples using the Batch CG solver.
#         cg_solver = BatchConjugateGradients(
#             A_apply_function=Afun_batch,
#             b=b_batch,
#             x0=torch.zeros_like(b_batch),
#             tol=1e-6,
#             early_stopping=False
#         )
#         beta_batch = cg_solver.solve()  # (trace_samples, M)
        
#         # Compute F @ (ws * beta_batch) for each trace sample.
#         D_beta_batch = ws * beta_batch  # (trace_samples, M)
#         F_expanded = F.unsqueeze(0).expand(trace_samples, -1, -1)  # (trace_samples, N, M)
#         F_beta = torch.bmm(F_expanded, D_beta_batch.unsqueeze(-1)).squeeze(-1)  # (trace_samples, N)
        
#         # Compute the corresponding alpha for each trace sample:
#         # alpha_batch = 1/sigmasq * (temp_rhs - F @ (ws * beta_batch))
#         temp_rhs_flat = temp_rhs_batch.squeeze(-1)  # (trace_samples, N)
#         alpha_batch = 1/sigmasq * (temp_rhs_flat - F_beta)  # (trace_samples, N)
        
#         # Now, compute the dot product z^T * alpha for each trace sample and average.
#         dots = torch.sum(Z * alpha_batch, dim=1)  # (trace_samples,)
#         term1[i] = torch.mean(dots)

#     ##############################################
#     # Final Hyperparameter Gradients
#     ##############################################
#     grad = 0.5 * (term1 - term2)
#     grad = grad.real  # if only the real parts are desired
#     # print("Gradient for hyperparameters:", grad)

#     return grad
# grad = efgp1d_gradient_batched(x, y, sigmasq, kernel,EPSILON,trace_samples=10)
# for i in range(grad.shape[0]):
#     print(f"Hyperparameter {i}: gradient approx = {grad[i].real.item():.6f}")

# Using flatiron NUFFT

In [ ]:
# def efgp1d_gradient_batched(x, y, sigmasq, kernel,eps,trace_samples,x0,x1):
#     n_hypers = 3  # number of hyperparameters

#     # Ensure x and y are float64 and flattened.
#     # x = x.to(dtype=torch.float64).flatten()   # shape: (N,)
#     # y = y.to(dtype=torch.float64).flatten()     # shape: (N,)

#     # x0 = xmin
#     # x1 = xmax
#     L = x1 - x0
#     N = x.shape[0]

#     # Get Fourier frequencies and quadrature weights.
#     xis, h, mtot = get_xis(kernel, eps, L)  # e.g. len(xis)==M, where M=97 for example.

#     # Compute weights: ws = sqrt(S(xis) * h)
#     ws = torch.sqrt(kernel.spectral_density(xis) * h)  # shape: (M,)
#     # We represent D as a diagonal using ws.
#     # Convert to complex type.
#     ws = ws.to(dtype=torch.complex128)
#     v = compute_convolution_vector_vectorized(m=int((mtot - 1) / 2), x=x, h=h)
#     Afun = lambda beta: ws * BatchFFTConv1d(v, ws * beta)() + sigmasq * beta #O(N + MlogM)?



#     #via NUFFT 
#     # xcen = (x1+x0)/2
#     xcen = 0
#     tphx = 2*math.pi*h*(x - xcen)
#     # tphxtrgs = 2*math.pi*h*(x - xcen)


#     nuffttol = 1e-15
#     isign = -1

#     # no matter what we incur a cost of O(N + MlogM) here, but would be faster to plan outside the training loop


#     fadjbeta = lambda beta: torch.tensor(finufft.nufft1d1(x=tphx.numpy(),
#                                                            c=beta.to(torch.complex128).numpy(),
#                                                            isign= isign, eps= nuffttol, n_modes=mtot)) #O(N+ MlogM)?
#         ##NUFFT version
#     fbeta = lambda beta: torch.tensor(
#     finufft.nufft1d2(
#         x=tphx.numpy(),
#         f=beta.to(torch.complex128).numpy(),
#         isign=1,
#         eps=nuffttol,
#         # n_modes=mtot
#     )
#     )


#     rhs = fadjbeta(y)
#     # print(rhs.shape)
#     rhs = ws * rhs       


#     cg_object = ConjugateGradients(
#         A_apply_function=Afun,
#         b=rhs,
#         x0=torch.zeros_like(rhs),
#         early_stopping=False
#         # early_stopping=opts.get('early_stopping', False)
#     )
#     beta = cg_object.solve()  # beta: (M,)
#     alpha = 1/sigmasq * (y - fbeta( ws * beta))  # alpha: (N,)

#     # -------------------
#     Dprime = h * kernel.spectral_grad(xis)  # shape: (M, n_params)
#     # This yields the derivative of the diagonal of D^2, since D^2 = diag(S(xis)*h).
#     term2 = torch.zeros(n_hypers, dtype=torch.complex128)
#     fadjalpha = fadjbeta(alpha)

#     for i in range(3-1):
#         ## alpha^* F D' F^* alpha
#         term2[i] = fadjalpha.conj()@ (Dprime[:,i]*fadjalpha)
#     # term2[i] = torch.matmul(alpha.conj().unsqueeze(0),
#     #                         torch.matmul(khat_prime[:, :, i], alpha.unsqueeze(1))).squeeze()
#     term2[-1] = alpha.conj() @ alpha  # noise term
#     # -------------------
#     # Compute term 1 (trace term) via Monte Carlo.
#     # trace_samples = 10
#     # try with rademachers instead..

#     ##############################################
#     # Monte Carlo Trace Term (term1) using Batch CG solver
#     ##############################################
#     # Create trace_samples Rademacher vectors. (Each entry is +1 or -1.)
#     Z = torch.randint(0, 2, (trace_samples, N), dtype=torch.float64, device=x.device) * 2 - 1  # shape: (trace_samples, N)
#     Z = Z.to(dtype=torch.complex128)  # cast to complex for consistency

#     # Define a batched operator based on FFTConv1d for use in the CG solver.
#     def Afun_batch(beta):
#         """
#         Batched operator acting on beta of shape (B, M).
#         Returns: ws * FFTConv1d(v, ws * beta)() + sigmasq * beta, shape (B, M).
#         """
#         # Multiply beta by ws elementwise (equivalent to a diagonal D multiplication)
#         D_beta = ws * beta
#         # FFTConv1d expects the second argument to have shape (B, L); ensure beta is batched.
#         conv_result = BatchFFTConv1d(v, D_beta)()  # Output shape: (B, M) if beta is (B, M)
#         return ws * conv_result + sigmasq * beta

#     B_list       = []
#     temp_rhs_list = []
#     p = 3
#     #TODO this is dependent on the kernel.. 
#     fadjz = fadjbeta(Z)
#     for i in range(p):
#         if i < p-1:
#             # term‑1 for hyper i
#             trhs = fbeta( ( fadjz * Dprime[:, i]) )
#         else:
#             # noise‑term has RHS = Z itself
#             trhs = Z                 # shape (T, N)
#         temp_rhs_list.append(trhs)

#         Bi = fadjbeta(trhs) * ws     # shape (T, M)
#         B_list.append(Bi)

#     # 2) Concatenate into one big batched RHS of shape (p*T, M)
#     B_all = torch.cat(B_list, dim=0)         # (p*T, M)
#     TR_all = torch.cat(temp_rhs_list, dim=0) # (p*T, N)

#     # 3) One single batched CG solve
#     cg = BatchConjugateGradients(
#         A_apply_function=Afun_batch,
#         b=B_all,
#         x0=torch.zeros_like(B_all),
#         tol=1e-6,
#         early_stopping=False
#     )
#     Beta_all = cg.solve()      # shape (p*T, M)

#     # 4) Recover the alpha’s in one go
#     #    α = (temp_rhs - F β·ws) / σ²
#     WS = ws.unsqueeze(0)               # shape (1, M)
#     FB = fbeta(Beta_all * WS)          # (p*T, N)
#     Alpha_all = (TR_all - FB) / sigmasq  # (p*T, N)

#     # 5) Split back into p blocks of size T
#     Alpha_blocks = Alpha_all.chunk(p, dim=0)  # list of p tensors, each (T, N)

#     # 6) Compute term1[i] = mean over T samples of <Z, α_i>
#     term1 = torch.empty(p, dtype=torch.complex128, device=x.device)
#     for i, α_block in enumerate(Alpha_blocks):
#         # Z is (T, N); α_block is (T, N)
#         dots = (Z * α_block).sum(dim=1)       # (T,)
#         term1[i] = dots.mean()

#     # … then term2 as before, and grad = 0.5*(term1 - term2)


#     # final gradients 
#     grad = 0.5 * (term1 - term2)
#     grad = grad.real  # if only the real parts are desired
    
#     # print("Gradient for hyperparameters:", grad)

#     return grad
# grad = efgp1d_gradient_batched(x, y, sigmasq, kernel,EPSILON,trace_samples=10,x0=x0,x1=x1)
# for i in range(grad.shape[0]):
#     print(f"Hyperparameter {i}: gradient approx = {grad[i].real.item():.6f}")


# previous gradient version

In [ ]:
# def efgp1d_gradient_batched(x, y, sigmasq, kernel,eps,trace_samples,x0,x1):
#     n_hypers = 3  # number of hyperparameters

#     # Ensure x and y are float64 and flattened.
#     # x = x.to(dtype=torch.float64).flatten()   # shape: (N,)
#     # y = y.to(dtype=torch.float64).flatten()     # shape: (N,)

#     # x0 = xmin
#     # x1 = xmax
#     L = x1 - x0
#     N = x.shape[0]

#     # Get Fourier frequencies and quadrature weights.
#     xis, h, mtot = get_xis(kernel, eps, L)  # e.g. len(xis)==M, where M=97 for example.

#     # Compute weights: ws = sqrt(S(xis) * h)
#     ws = torch.sqrt(kernel.spectral_density(xis) * h)  # shape: (M,)
#     # We represent D as a diagonal using ws.
#     # Convert to complex type.
#     ws = ws.to(dtype=torch.complex128)
#     v = compute_convolution_vector_vectorized(m=int((mtot - 1) / 2), x=x, h=h)
#     Afun = lambda beta: ws * BatchFFTConv1d(v, ws * beta)() + sigmasq * beta #O(N + MlogM)?



#     #via NUFFT 
#     # xcen = (x1+x0)/2
#     xcen = 0
#     tphx = 2*math.pi*h*(x - xcen)
#     # tphxtrgs = 2*math.pi*h*(x - xcen)


#     nuffttol = 1e-15
#     isign = -1

#     # no matter what we incur a cost of O(N + MlogM) here, but would be faster to plan outside the training loop
#     fadjbeta = lambda beta: torch.tensor(finufft.nufft1d1(x=tphx.numpy(),
#                                                            c=beta.to(torch.complex128).numpy(),
#                                                            isign= isign, eps= nuffttol, n_modes=mtot)) #O(N+ MlogM)?
#         ##NUFFT version
#     fbeta = lambda beta: torch.tensor(
#     finufft.nufft1d2(
#         x=tphx.numpy(),
#         f=beta.to(torch.complex128).numpy(),
#         isign=1,
#         eps=nuffttol,
#         # n_modes=mtot
#     )
# )
#     rhs = fadjbeta(y)
#     rhs = ws * rhs       


#     cg_object = ConjugateGradients(
#         A_apply_function=Afun,
#         b=rhs,
#         x0=torch.zeros_like(rhs),
#         early_stopping=False
#         # early_stopping=opts.get('early_stopping', False)
#     )
#     beta = cg_object.solve()  # beta: (M,)
#     alpha = 1/sigmasq * (y - fbeta( ws * beta))  # alpha: (N,)

#     # -------------------
#     Dprime = h * kernel.spectral_grad(xis)  # shape: (M, n_params)
#     # This yields the derivative of the diagonal of D^2, since D^2 = diag(S(xis)*h).
#     term2 = torch.zeros(n_hypers, dtype=torch.complex128)
#     for i in range(3-1):
#         ## alpha^* F D' F^* alpha
#         fadjalpha = fadjbeta(alpha)
#         term2[i] = fadjalpha.conj()@ (Dprime[:,i]*fadjalpha)
#     # term2[i] = torch.matmul(alpha.conj().unsqueeze(0),
#     #                         torch.matmul(khat_prime[:, :, i], alpha.unsqueeze(1))).squeeze()
#     term2[-1] = alpha.conj() @ alpha  # noise term
#     # -------------------
#     # Compute term 1 (trace term) via Monte Carlo.
#     # trace_samples = 10
#     # try with rademachers instead..

#     ##############################################
#     # Monte Carlo Trace Term (term1) using Batch CG solver
#     ##############################################
#     # Create trace_samples Rademacher vectors. (Each entry is +1 or -1.)
#     Z = torch.randint(0, 2, (trace_samples, N), dtype=torch.float64, device=x.device) * 2 - 1  # shape: (trace_samples, N)
#     Z = Z.to(dtype=torch.complex128)  # cast to complex for consistency

#     # Define a batched operator based on FFTConv1d for use in the CG solver.
#     def Afun_batch(beta):
#         """
#         Batched operator acting on beta of shape (B, M).
#         Returns: ws * FFTConv1d(v, ws * beta)() + sigmasq * beta, shape (B, M).
#         """
#         # Multiply beta by ws elementwise (equivalent to a diagonal D multiplication)
#         D_beta = ws * beta
#         # FFTConv1d expects the second argument to have shape (B, L); ensure beta is batched.
#         conv_result = BatchFFTConv1d(v, D_beta)()  # Output shape: (B, M) if beta is (B, M)
#         return ws * conv_result + sigmasq * beta

#     term1 = torch.zeros(n_hypers, dtype=torch.complex128)

#     # will have to change for different kernels.. 
#     for i in range(n_hypers):
#         if i<n_hypers-1:
#             temp_rhs = (fbeta((fadjbeta(Z) * Dprime[:,i])))
#         else:
#             temp_rhs = Z
#         b_batch = (fadjbeta(temp_rhs) * ws)
#         cg_solver = BatchConjugateGradients(
#             A_apply_function=Afun_batch,
#             b=b_batch,
#             x0=torch.zeros_like(b_batch),
#             tol=1e-6,
#             early_stopping=False
#         )
#         beta_batch = cg_solver.solve()  # (trace_samples, M)
#         alpha_batch = 1/sigmasq * (temp_rhs - (fbeta(beta_batch*ws)))
#         dots = torch.sum(Z * alpha_batch, dim=1) 
#         term1[i] = torch.mean(dots)

#     # final gradients 
#     grad = 0.5 * (term1 - term2)
#     grad = grad.real  # if only the real parts are desired
    
#     # print("Gradient for hyperparameters:", grad)

#     return grad
# grad = efgp1d_gradient_batched(x, y, sigmasq, kernel,EPSILON,trace_samples=10,x0=x0,x1=x1)
# for i in range(grad.shape[0]):
#     print(f"Hyperparameter {i}: gradient approx = {grad[i].real.item():.6f}")


# Variance stuff

In [ ]:
# import torch
# import math
# import matplotlib.pyplot as plt

# # --- Assumed Inputs (Define these *before* this block) ---
# # x: Training inputs (N, d) tensor, device=device, dtype=rdtype
# # y: Training outputs (N,) tensor, device=device, dtype=rdtype
# # x_new: Test inputs (B, d) tensor, device=device, dtype=rdtype
# # xis: Frequency points (M, d) tensor, device=device, dtype=rdtype
# # ws: Spectral weights sqrt(density*h^d) (M,) tensor, device=device, dtype=cdtype
# # sigmasq: Noise variance (scalar float)
# # variance_direct: Pre-calculated exact variance (B,) tensor, device=device, dtype=rdtype
# # BatchConjugateGradients: Class or function for batched CG solves
# # device: torch.device
# # rdtype: torch.float64
# # cdtype: torch.complex128
# # -----------------------------------------------------------

# # --- Self-Contained Hutchinson Block ---

# # --- Setup ---
# print("Starting Hutchinson Estimation Block...")
# N, d = x.shape
# B = x_new.shape[0]
# M = xis.shape[0]
# if d != 1:
#     print(f"Warning: Hutchinson variance estimation is implemented assuming d=1 for final summation.")
#     # Or raise NotImplementedError("Hutchinson diagonal sum variance estimation requires d=1.")

# # Device/Type Setup (Redundant if defined globally, but safe)
# device = x.device
# rdtype = torch.float64
# cdtype = torch.complex128

# # Ensure inputs are correct type/device (Redundant if done globally)
# x = x.to(device, rdtype)
# y = y.to(device, rdtype)
# x_new = x_new.to(device, rdtype)
# xis = xis.to(device, rdtype)
# ws = ws.to(device, cdtype)
# sigmasq_scalar = float(sigmasq) # Use scalar float for division/addition alpha

# # Recompute F_train (needed for A)
# print("Computing F_train...")
# F_train = torch.exp(2 * math.pi * 1j * torch.matmul(x, xis.T)).to(cdtype)

# # Define D (Diagonal matrix from weights)
# print("Defining D...")
# D = torch.diag(ws) # Assuming ws components are positive sqrt

# # # Define h (Frequency spacing - Crucial for final summation step)
# # h = torch.tensor(0.0, device=device, dtype=rdtype) # Initialize
# # if M > 1 and d == 1:
# #    # Calculate spacing from the first dimension assuming uniform grid
# #    h = (xis[1, 0] - xis[0, 0]).abs()
# #    print(f"Calculated frequency spacing h = {h.item()}")
# # elif M > 1 and d > 1:
# #    # Assume spacing is the same in all dimensions for simplicity? Or take first dim?
# #    h = (xis[1, 0] - xis[0, 0]).abs() # Using first dim as convention
# #    print(f"Warning: Using h from first dimension for d={d}. h = {h.item()}")
# # else: # M <= 1
# #    h = torch.tensor(1.0, device=device, dtype=rdtype) # Placeholder if M=1
# #    print(f"Warning: M={M}. Using placeholder h=1.0")
# # if h <= 1e-12:
# #     print(f"Warning: Calculated h ({h.item()}) is very small or zero. Check xis.")
# #     h = torch.tensor(1.0, device=device, dtype=rdtype) # Reset to avoid division issues

# # Create identity matrix for M dimensions
# I_M = torch.eye(M, device=device, dtype=cdtype) # Use cdtype for complex arithmetic

# # 1) Re-build A = (D F^H F D)/σ^2 + I_M
# print("Building matrix A...")
# F_conj_T = F_train.conj().T
# # Ensure matrix multiplications are correct device/type
# A = (D @ F_conj_T @ F_train @ D) / sigmasq_scalar + I_M
# print(f"Matrix A shape: {A.shape}, dtype: {A.dtype}")

# # --- Hutchinson-type Estimation Block ---

# # 2) draw J Rademacher probes with length M
# J = 5000 # Number of probes
# print(f"Generating {J} Rademacher probes...")
# # Sample ±1 i.i.d. (Rademacher), then cast to complex
# etas = (torch.randint(0, 2, (J, M), device=device, dtype=rdtype) * 2 - 1).to(dtype=cdtype)  # Shape (J, M)

# # 3) solve A u = D η via batched‐CG, then set γ = D u
# print("Calculating RHS for CG...")
# rhs = (D @ etas.T).T # Shape (J, M)

# # Define the batched matrix-vector product for A
# def A_apply_batched(x):
#     # x is (J, M). A is (M, M).
#     return x @ A # (J, M) @ (M, M) -> (J, M)

# # Use a very tight tolerance
# cg_tol = 1e-6 # TIGHT tolerance
# max_cg_iter = M * 5 # Increase iterations further if needed

# print(f"Running Batch CG (tol={cg_tol}, max_iter={max_cg_iter})...")
# cg = BatchConjugateGradients(
#     A_apply_batched, rhs, 
#     tol=cg_tol,
#     max_iter=max_cg_iter,
#     x0=torch.zeros_like(rhs), # Use complex zeros
#     early_stopping=True
# )
# us = cg.solve() # shape (J, M) solves A u = D eta

# print("Calculating gammas...")
# gammas = (D @ us.T).T # shape (J, M) approximates C^{-1} @ etas

# # 4) form Hutchinson estimates of the diagonal sums Sum_i Cinv[i, i+k]
# print("Estimating diagonal sums...")
# offsets = list(range(-(M-1), M))
# num_offsets = len(offsets)
# est_sums = torch.zeros(num_offsets, device=device, dtype=cdtype)

# # Precompute conjugate probes
# etas_conj = etas.conj()

# # --- Vectorized calculation over probes J ---
# for i, k in enumerate(offsets):
#     est_sum_val = torch.tensor(0.0 + 0.0j, device=device, dtype=cdtype)
#     if k == 0:
#         est_sum_val = torch.sum(etas_conj * gammas, dim=1).mean()
#     elif k > 0:
#         overlap = M - k
#         if overlap > 0:
#             est_sum_val = torch.sum(etas_conj[:, :overlap] * gammas[:, k:], dim=1).mean()
#     else:  # k < 0
#         k_abs = abs(k)
#         overlap = M - k_abs
#         if overlap > 0:
#             est_sum_val = torch.sum(etas_conj[:, k_abs:] * gammas[:, :overlap], dim=1).mean()
#     est_sums[i] = est_sum_val
# print("Diagonal sum estimation complete.")

# # 5) Diagnostics (Optional - Compare est_sums to true_sums)
# print("Performing diagnostics on diagonal sums...")
# try:
#     with torch.no_grad(): # Avoid tracking gradients for inversion/diagonals
#         D_inv = torch.diag(1.0 / torch.diag(D))
#         C0 = D_inv @ A @ D_inv
#         Cinv = torch.linalg.inv(C0)
#         true_sums = torch.zeros(len(offsets), device=device, dtype=cdtype)
#         for i_diag, k_diag in enumerate(offsets):
#             true_sums[i_diag] = Cinv.diagonal(offset=k_diag).sum().to(cdtype)

#         err_diag_sums = est_sums - true_sums
#         rel_errors = err_diag_sums / true_sums
#         print(f"Max Abs Error (est_sums vs true_sums): {err_diag_sums.abs().max().item():.2e}")
#         print(f"RMSE (est_sums vs true_sums):        {torch.sqrt((err_diag_sums.abs()**2).mean()).item():.2e}")

#         plt.figure(figsize=(15, 5))
#         plt.subplot(1, 3, 1)
#         plt.plot(offsets, true_sums.real.cpu().numpy(), 'bo-', ms=4, label="True Sums (Real)")
#         plt.plot(offsets, est_sums.real.cpu().numpy(), 'rx--', ms=4, label="Est Sums (Real)")
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("Sum Value (Real Part)")
#         plt.title("Comparison of Diagonal Sums")
#         plt.legend()
#         plt.grid(True)

#         plt.subplot(1, 3, 2)
#         plt.plot(offsets, err_diag_sums.abs().cpu().numpy(), '.-')
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("|Error|")
#         plt.title("Magnitude of Error")
#         plt.yscale('log')
#         plt.grid(True)

#         plt.subplot(1, 3, 3)
#         plt.plot(offsets, rel_errors.abs().cpu().numpy(), '.-')
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("Relative Error")
#         plt.title("Relative Error (Log Scale)")
#         plt.yscale('log')
#         plt.grid(True)
#         plt.tight_layout()
#         plt.show()
# except Exception as e:
#     print(f"Could not compute true_sums or diagnostics: {e}")
#     true_sums = None

# # 6) Build the difference‐grid and frequency weights (assuming d=1)
# print("Building final summation weights...")
# xi_diff = torch.tensor(offsets, device=device, dtype=rdtype) * h  # (num_offsets,)

# x_new_1d = x_new.squeeze()
# if x_new_1d.dim() != 1:
#      raise ValueError(f"x_new could not be squeezed to 1D. Original shape: {x_new.shape}")

# exponent_arg = -2j * math.pi * (x_new_1d.unsqueeze(1) * xi_diff.unsqueeze(0))
# expo_diff = torch.exp(exponent_arg).to(dtype=cdtype)

# # 7) Form the final variance estimate s_est = Re(expo_diff @ est_sums)
# print("Calculating final variance estimate s_est...")
# s_est = (expo_diff @ est_sums).real
# s_est = torch.clamp_min(s_est, 0.0)

# # 8) Final Comparison Plot
# all_vals = torch.cat((s_est.cpu(), variance_direct.cpu())).numpy()
# min_val = min(0, all_vals.min()) * 1.1
# max_val = max(0, all_vals.max()) * 1.1
# lims = [min_val, max_val]

# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(lims, lims, 'r--', alpha=0.7, label='y=x')
# plt.scatter(s_est.cpu().numpy(), variance_direct.cpu().numpy(), label="Hutchinson (s_est)", s=20)
# plt.xlabel("s_est (Hutchinson Estimate)")
# plt.ylabel("variance_direct")
# plt.title("Variance Comparison (Cleaned Hutchinson)")
# plt.legend()
# plt.grid(True)
# plt.xlim(lims)
# plt.ylim(lims)

# plt.subplot(1, 2, 2)
# plt.plot(x_new_1d.cpu().numpy(), s_est.cpu().numpy(), 'b.-', label='s_est (Hutchinson)')
# plt.plot(x_new_1d.cpu().numpy(), variance_direct.cpu().numpy(), 'r.-', label='variance_direct')
# plt.xlabel("x_new")
# plt.ylabel("Variance")
# plt.title("Variance Estimates vs. Position")
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()

# print("Hutchinson Estimation Block Finished.")

In [ ]:
# import torch
# import math
# import matplotlib.pyplot as plt

# # --- Assumed Inputs (Define these *before* this block) ---
# # x: Training inputs (N, d) tensor, device=device, dtype=rdtype
# # y: Training outputs (N,) tensor, device=device, dtype=rdtype
# # x_new: Test inputs (B, d) tensor, device=device, dtype=rdtype
# # xis: Frequency points (M, d) tensor, device=device, dtype=rdtype
# # ws: Spectral weights sqrt(density*h^d) (M,) tensor, device=device, dtype=cdtype
# # sigmasq: Noise variance (scalar float)
# # variance_direct: Pre-calculated exact variance (B,) tensor, device=device, dtype=rdtype
# # BatchConjugateGradients: Class or function for batched CG solves
# # device: torch.device
# # rdtype: torch.float64
# # cdtype: torch.complex128
# # -----------------------------------------------------------

# # --- Self-Contained Hutchinson Block ---

# # --- Setup ---
# print("Starting Hutchinson Estimation Block...")
# N, d = x.shape
# B = x_new.shape[0]
# M = xis.shape[0]
# if d != 1:
#     print(f"Warning: Hutchinson variance estimation is implemented assuming d=1 for final summation.")
#     # Or raise NotImplementedError("Hutchinson diagonal sum variance estimation requires d=1.")

# # Device/Type Setup (Redundant if defined globally, but safe)
# device = x.device
# rdtype = torch.float64
# cdtype = torch.complex128

# # Ensure inputs are correct type/device (Redundant if done globally)
# x = x.to(device, rdtype)
# y = y.to(device, rdtype)
# x_new = x_new.to(device, rdtype)
# xis = xis.to(device, rdtype)
# ws = ws.to(device, cdtype)
# sigmasq_scalar = float(sigmasq) # Use scalar float for division/addition alpha

# # Recompute F_train (needed for A)
# print("Computing F_train...")
# F_train = torch.exp(2 * math.pi * 1j * torch.matmul(x, xis.T)).to(cdtype)

# # Define D (Diagonal matrix from weights)
# print("Defining D...")
# D = torch.diag(ws) # Assuming ws components are positive sqrt

# # Define h (Frequency spacing - Crucial for final summation step)
# # h = torch.tensor(0.0, device=device, dtype=rdtype) # Initialize
# # if M > 1 and d == 1:
# #    # Calculate spacing from the first dimension assuming uniform grid
# #    h = (xis[1, 0] - xis[0, 0]).abs()
# #    print(f"Calculated frequency spacing h = {h.item()}")
# # elif M > 1 and d > 1:
# #    # Assume spacing is the same in all dimensions for simplicity? Or take first dim?
# #    h = (xis[1, 0] - xis[0, 0]).abs() # Using first dim as convention
# #    print(f"Warning: Using h from first dimension for d={d}. h = {h.item()}")
# # else: # M <= 1
# #    h = torch.tensor(1.0, device=device, dtype=rdtype) # Placeholder if M=1
# #    print(f"Warning: M={M}. Using placeholder h=1.0")
# # if h <= 1e-12:
# #     print(f"Warning: Calculated h ({h.item()}) is very small or zero. Check xis.")
# #     h = torch.tensor(1.0, device=device, dtype=rdtype) # Reset to avoid division issues

# # Create identity matrix for M dimensions
# I_M = torch.eye(M, device=device, dtype=cdtype) # Use cdtype for complex arithmetic

# # 1) Re-build A = (D F^H F D)/σ^2 + I_M
# print("Building matrix A...")
# F_conj_T = F_train.conj().T
# # Ensure matrix multiplications are correct device/type
# A = (D @ F_conj_T @ F_train @ D) / sigmasq_scalar + I_M
# print(f"Matrix A shape: {A.shape}, dtype: {A.dtype}")

# # --- Hutchinson-type Estimation Block ---

# # 2) draw J Rademacher probes with length M
# J = 10000 # Number of probes
# print(f"Generating {J} Rademacher probes...")
# # Sample ±1 i.i.d. (Rademacher), then cast to complex
# etas = (torch.randint(0, 2, (J, M), device=device, dtype=rdtype) * 2 - 1).to(dtype=cdtype)  # Shape (J, M)

# # 3) solve A u = D η via batched‐CG, then set γ = D u
# print("Calculating RHS for CG...")
# rhs = (D @ etas.T).T # Shape (J, M)

# # Define the batched matrix-vector product for A
# def A_apply_batched(x):
#     # x is (J, M). A is (M, M).
#     return x @ A # (J, M) @ (M, M) -> (J, M)







# print(f"Running Batch CG (tol={cg_tol}, max_iter={max_cg_iter})...")
# cg = BatchConjugateGradients(
#     A_apply_batched, rhs, 
#     tol=cg_tol,
#     max_iter=max_cg_iter,
#     x0=torch.zeros_like(rhs), # Use complex zeros
#     early_stopping=True
# )
# us = cg.solve() # shape (J, M) solves A u = D eta

# print("Calculating gammas...")
# gammas = (D @ us.T).T # shape (J, M) approximates C^{-1} @ etas

# # 4) form Hutchinson estimates of the diagonal sums Sum_i Cinv[i, i+k]
# print("Estimating diagonal sums...")
# offsets = list(range(-(M-1), M))
# num_offsets = len(offsets)
# est_sums = torch.zeros(num_offsets, device=device, dtype=cdtype)

# # Precompute conjugate probes
# etas_conj = etas.conj()

# # --- Vectorized calculation over probes J ---
# # 4) form Hutchinson estimates of the diagonal sums via one FFT-based convolution
# print("Estimating diagonal sums via FFT-based convolution...")

# # zero-pad both gammas and etas to length L >= 2*M-1 so that circular conv = linear conv
# L = 1 << ((2*M - 1).bit_length())           # next power of two ≥ 2*M-1
# pad = L - M

# # (J, M) → (J, L)
# gammas_pad = torch.cat([gammas, torch.zeros(J, pad, dtype=cdtype, device=device)], dim=1)
# etas_pad   = torch.cat([etas_conj, torch.zeros(J, pad, dtype=cdtype, device=device)], dim=1)

# # FFTs (O(L log L) each)
# G = torch.fft.fft(gammas_pad, n=L)         # (J, L//2+1)
# E = torch.fft.fft(etas_pad,   n=L)         # (J, L//2+1)

# # pointwise product → iFFT to get full linear cross-correlation of length L
# R = torch.fft.ifft(G * torch.conj(E), n=L)


# # 1) positive-and-zero lags:  j = 0,1,...,M-1
# pos = R[:, :M]                        # shape (J, M)

# # 2) negative lags: j = -(M-1),..., -1  →  indices L-(M-1) ... L-1
# neg = R[:, L-(M-1) : L]               # shape (J, M-1)

# # 3) concatenate so that rags[:, 0] = c_{-(M-1)}, ... , rags[:, M-1] = c_0, ... rags[:, 2M-2] = c_{M-1}
# rags = torch.cat([neg, pos], dim=1)   # shape (J, 2*M-1)

# # 4) average over the J Monte–Carlo samples
# est_sums = rags.mean(dim=0)           # (2*M-1,)
# # making symmetric

# ## double check this 
# # est_sums = (est_sums + est_sums.flip(0)) * 0.5

# print("Diagonal sum estimation complete.")


# # 5) Diagnostics (Optional - Compare est_sums to true_sums)
# print("Performing diagnostics on diagonal sums...")
# try:
#     with torch.no_grad(): # Avoid tracking gradients for inversion/diagonals
#         D_inv = torch.diag(1.0 / torch.diag(D))
#         C0 = D_inv @ A @ D_inv
#         Cinv = torch.linalg.inv(C0)
#         true_sums = torch.zeros(len(offsets), device=device, dtype=cdtype)
#         for i_diag, k_diag in enumerate(offsets):
#             true_sums[i_diag] = Cinv.diagonal(offset=k_diag).sum().to(cdtype)

#         err_diag_sums = est_sums - true_sums
#         rel_errors = err_diag_sums / true_sums
#         print(f"Max Abs Error (est_sums vs true_sums): {err_diag_sums.abs().max().item():.2e}")
#         print(f"RMSE (est_sums vs true_sums):        {torch.sqrt((err_diag_sums.abs()**2).mean()).item():.2e}")

#         plt.figure(figsize=(15, 5))
#         plt.subplot(1, 3, 1)
#         plt.plot(offsets, true_sums.real.cpu().numpy(), 'bo-', ms=4, label="True Sums (Real)")
#         plt.plot(offsets, est_sums.real.cpu().numpy(), 'rx--', ms=4, label="Est Sums (Real)")
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("Sum Value (Real Part)")
#         plt.title("Comparison of Diagonal Sums")
#         plt.legend()
#         plt.grid(True)

#         plt.subplot(1, 3, 2)
#         plt.plot(offsets, err_diag_sums.abs().cpu().numpy(), '.-')
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("|Error|")
#         plt.title("Magnitude of Error")
#         plt.yscale('log')
#         plt.grid(True)

#         plt.subplot(1, 3, 3)
#         plt.plot(offsets, rel_errors.abs().cpu().numpy(), '.-')
#         plt.xlabel("Diagonal offset k")
#         plt.ylabel("Relative Error")
#         plt.title("Relative Error (Log Scale)")
#         plt.yscale('log')
#         plt.grid(True)
#         plt.tight_layout()
#         plt.show()
# except Exception as e:
#     print(f"Could not compute true_sums or diagnostics: {e}")
#     true_sums = None

# # 6) Build the difference‐grid and frequency weights (assuming d=1)
# print("Building final summation weights...")
# xi_diff = torch.tensor(offsets, device=device, dtype=rdtype) * h  # (num_offsets,)
# est_sums = est_sums.contiguous()          # complex tensor, shape (N,)
# assert est_sums.shape[0] == xi_diff.shape[0], "length mismatch!"

# N = est_sums.shape[0]       # total modes (25)
# K = (N - 1) // 2            # highest ±k    (12)

# # ------------------------------------------------------------
# # 1.  direct reference result (for sanity check)
# # ------------------------------------------------------------
# # x_new_1d = x_new.squeeze()               # shape (B,)
# # expo = torch.exp(
# #         -2j * math.pi * (x_new_1d[:,None] * xi_diff[None,:])
# #       ).to(dtype=cdtype)
# # s_direct = (expo @ est_sums).real        # shape (B,)

# # ------------------------------------------------------------
# # 2.  NUFFT path – put coeffs in FFT order, call FINUFFT
# # ------------------------------------------------------------
# fk_fft = torch.roll(est_sums, -K)        # CMCL → FFT order (ifftshift)

# phi = (-2 * math.pi * h * (x_new_1d - xcen)).unsqueeze(0)  # shape (1,B)

# s_est_complex = pff.finufft_type2(
#         phi, fk_fft, eps=nufft_eps, isign=1, modeord=True  # FFT order
# )
# s_nufft = s_est_complex.real
# # print("max |difference|:", (s_nufft - s_direct).abs().max())


# # 7) Form the final variance estimate s_est = Re(expo_diff @ est_sums)
# print("Calculating final variance estimate s_est...")
# # x_new_1d = x_new.squeeze()
# # if x_new_1d.dim() != 1:
# #      raise ValueError(f"x_new could not be squeezed to 1D. Original shape: {x_new.shape}")

# # exponent_arg = -2j * math.pi * (x_new_1d.unsqueeze(1) * xi_diff.unsqueeze(0))
# # expo_diff = torch.exp(exponent_arg).to(dtype=cdtype)

# # # 7) Form the final variance estimate s_est = Re(expo_diff @ est_sums)
# # print("Calculating final variance estimate s_est...")
# # s_est = (expo_diff @ est_sums).real
# # s_est = torch.clamp_min(s_est, 0.0)
# s_est = s_nufft
# # 8) Final Comparison Plot
# all_vals = torch.cat((s_est.cpu(), variance_direct.cpu())).numpy()
# min_val = min(0, all_vals.min()) * 1.1
# max_val = max(0, all_vals.max()) * 1.1
# lims = [min_val, max_val]

# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(lims, lims, 'r--', alpha=0.7, label='y=x')
# plt.scatter(s_est.cpu().numpy(), variance_direct.cpu().numpy(), label="Hutchinson (s_est)", s=20)
# plt.xlabel("s_est (Hutchinson Estimate)")
# plt.ylabel("variance_direct")
# plt.title("Variance Comparison (Cleaned Hutchinson)")
# plt.legend()
# plt.grid(True)
# plt.xlim(lims)
# plt.ylim(lims)

# plt.subplot(1, 2, 2)
# plt.plot(x_new_1d.cpu().numpy(), s_est.cpu().numpy(), 'b.-', label='s_est (Hutchinson)')
# plt.plot(x_new_1d.cpu().numpy(), variance_direct.cpu().numpy(), 'r.-', label='variance_direct')
# plt.xlabel("x_new")
# plt.ylabel("Variance")
# plt.title("Variance Estimates vs. Position")
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()

# print("Hutchinson Estimation Block Finished.")

# sum ests in 1d old misaligned

In [ ]:
# ## 1d 


# # def diag_sums(A_apply_batched,J,M,max_cg_iter,cg_tol):
# #     ## Takes an operator A_apply_batched and returns an estimate of the diagonal sums of the inverse of A
# #     ## Using Hutchinson's method with offsets
# #     ## Solve Au = eta via batched CG, then set gamma = u
# #     ## Then use convolution of gamma with eta to get the diagonal sums of the inverse of A
# #     etas = (torch.randint(0, 2, (J, M), device=device, dtype=rdtype) * 2 - 1).to(dtype=cdtype)  # Shape (J, M)

# #     # 3) solve A u = D eta via batched‐CG, then set gamma= D u
# #     print("Calculating RHS for CG...")
# #     rhs = (ws * etas) # Shape (J, M)

# #     print(f"Running Batch CG (tol={cg_tol}, max_iter={max_cg_iter})...")
# #     cg = BatchConjugateGradients(
# #         A_apply_batched, rhs, 
# #         tol=cg_tol,
# #         max_iter=max_cg_iter,
# #         x0=torch.zeros_like(rhs), 
# #         early_stopping=True
# #     )
# #     us = cg.solve() # shape (J, M) solves A u = D eta

# #     print("Calculating gammas...")
# #     gammas = (ws * us) # shape (J, M) approximates C^{-1} @ etas

# #     # 4) form Hutchinson estimates of the diagonal sums Sum_i Cinv[i, i+k]
# #     print("Estimating diagonal sums...")
# #     offsets = list(range(-(M-1), M))
# #     num_offsets = len(offsets)
# #     est_sums = torch.zeros(num_offsets, device=device, dtype=cdtype)


# #     # --- Vectorized calculation over probes J ---
# #     # 4) form Hutchinson estimates of the diagonal sums via one FFT-based convolution
# #     print("Estimating diagonal sums via FFT-based convolution...")

# #     # zero-pad both gammas and etas to length L >= 2*M-1 so that circular conv = linear conv
# #     L = 1 << ((2*M - 1).bit_length())           # next power of two ≥ 2*M-1
# #     pad = L - M

# #     # (J, M) → (J, L)
# #     gammas_pad = torch.cat([gammas, torch.zeros(J, pad, dtype=cdtype, device=device)], dim=1)
# #     etas_pad   = torch.cat([etas, torch.zeros(J, pad, dtype=cdtype, device=device)], dim=1)

# #     # FFTs (O(L log L) each)
# #     G = torch.fft.fft(gammas_pad, n=L)         # (J, L//2+1)
# #     E = torch.fft.fft(etas_pad,   n=L)         # (J, L//2+1)

# #     # pointwise product → iFFT to get full linear cross-correlation of length L
# #     R = torch.fft.ifft(G * torch.conj(E), n=L)


# #     # 1) positive-and-zero lags:  j = 0,1,...,M-1
# #     pos = R[:, :M]                        # shape (J, M)

# #     # 2) negative lags: j = -(M-1),..., -1  →  indices L-(M-1) ... L-1
# #     neg = R[:, L-(M-1) : L]               # shape (J, M-1)

# #     # 3) concatenate so that rags[:, 0] = c_{-(M-1)}, ... , rags[:, M-1] = c_0, ... rags[:, 2M-2] = c_{M-1}
# #     rags = torch.cat([neg, pos], dim=1)   # shape (J, 2*M-1)

# #     # 4) average over the J Monte–Carlo samples
# #     est_sums = rags.mean(dim=0)           # (2*M-1,)
# #     return est_sums





# # est_sums = diag_sums(A_var_apply,J,M,max_cg_iter,cg_tol)

# if d==1:
#     x_new_1d = x_new.squeeze()               # shape (B,)

#     ## Plotting check
#     D_inv = torch.diag(1.0 / torch.diag(D))
#     A = (D @ F_conj_T @ F_train @ D) / sigmasq + I_M

#     C0 = D_inv @ A @ D_inv
#     Cinv = torch.linalg.inv(C0)




#     true_sums = torch.zeros(len(offsets), device=device, dtype=cdtype)
#     for i_diag, k_diag in enumerate(offsets):
#         true_sums[i_diag] = Cinv.diagonal(offset=k_diag).sum().to(cdtype)

#     err_diag_sums = est_sums - true_sums
#     rel_errors = err_diag_sums / true_sums
#     print(f"Max Abs Error (est_sums vs true_sums): {err_diag_sums.abs().max().item():.2e}")
#     print(f"RMSE (est_sums vs true_sums):        {torch.sqrt((err_diag_sums.abs()**2).mean()).item():.2e}")

#     plt.figure(figsize=(15, 5))
#     plt.subplot(1, 3, 1)
#     plt.plot(offsets, true_sums.real.cpu().numpy(), 'bo-', ms=4, label="True Sums (Real)")
#     plt.plot(offsets, est_sums.real.cpu().numpy(), 'rx--', ms=4, label="Est Sums (Real)")
#     plt.xlabel("Diagonal offset k")
#     plt.ylabel("Sum Value (Real Part)")
#     plt.title("Comparison of Diagonal Sums")
#     plt.legend()
#     plt.grid(True)

#     plt.subplot(1, 3, 2)
#     plt.plot(offsets, err_diag_sums.abs().cpu().numpy(), '.-')
#     plt.xlabel("Diagonal offset k")
#     plt.ylabel("|Error|")
#     plt.title("Magnitude of Error")
#     plt.yscale('log')
#     plt.grid(True)

#     plt.subplot(1, 3, 3)
#     plt.plot(offsets, rel_errors.abs().cpu().numpy(), '.-')
#     plt.xlabel("Diagonal offset k")
#     plt.ylabel("Relative Error")
#     plt.title("Relative Error (Log Scale)")
#     plt.yscale('log')
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()




# ## TODO Fix for 3D
# def nufft_var_est(est_sums,h,xcen,nufft_eps):

#     ## Takes an estimate of the diagonal sums of the inverse of A and returns an estimate of the variance
#     ## Using the NUFFT to compute \sum_{k\in [-M,M]} exp(2 \pi i k x) \hat{C}_{k}
#     # xi_diff = torch.tensor(offsets, device=device, dtype=rdtype) * h  # (num_offsets,)
#     # est_sums = est_sums.contiguous()          # complex tensor, shape (N,)
#     # assert est_sums.shape[0] == xi_diff.shape[0], "length mismatch!"

#     N = est_sums.shape[0]       # total modes (25)
#     K = (N - 1) // 2            # highest ±k    (12)

#     # ------------------------------------------------------------
#     # 1.  direct reference result (for sanity check)
#     # ------------------------------------------------------------
#     # x_new_1d = x_new.squeeze()               # shape (B,)
#     # expo = torch.exp(
#     #         -2j * math.pi * (x_new_1d[:,None] * xi_diff[None,:])
#     #       ).to(dtype=cdtype)
#     # s_direct = (expo @ est_sums).real        # shape (B,)

#     # ------------------------------------------------------------
#     # 2.  NUFFT path – put coeffs in FFT order, call FINUFFT
#     # ------------------------------------------------------------
#     # TODO understand the ordering of the FFT, for now just know it matches with the reference result
#     # TODO this might change in 3d, need to check
#     # fk_fft should be size 
#     fk_fft = torch.roll(est_sums, -K)        # CMCL → FFT order (ifftshift)

#     phi = (2 * math.pi * (h*(x_new - xcen)))  # shape (1,B)

#     s_est_complex = pff.finufft_type2(
#             phi.T, fk_fft, eps=nufft_eps, isign=1, modeord=True  # FFT order
#     )
#     s_nufft = s_est_complex.real
#     return s_nufft
# import torch, math

